In [1]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program, load_all_programs
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

In [21]:
import seaborn
import itertools
palette = seaborn.color_palette('pastel')[:5]
for c in range(1, 2+1):
    for idxs in itertools.combinations(range(len(palette)), c):
        colors = [f'rgb({round(r*255)}, {round(g*255)}, {round(b*255)})'
                  for (r, g, b) in [palette[i] for i in idxs]]
        incr = 10
        gradient = [f'{colors[i]} {i*incr}px, {colors[i]} {(i+1)*incr}px'
                    for i in range(len(colors))]
        print(f"""
.color-{"-".join(map(str, idxs))} {{
  background-image: repeating-linear-gradient(45deg, {", ".join(gradient)})
}}
""".strip())

.color-0 {
  background-image: repeating-linear-gradient(45deg, rgb(161, 201, 244) 0px, rgb(161, 201, 244) 10px)
}
.color-1 {
  background-image: repeating-linear-gradient(45deg, rgb(255, 180, 130) 0px, rgb(255, 180, 130) 10px)
}
.color-2 {
  background-image: repeating-linear-gradient(45deg, rgb(141, 229, 161) 0px, rgb(141, 229, 161) 10px)
}
.color-3 {
  background-image: repeating-linear-gradient(45deg, rgb(255, 159, 155) 0px, rgb(255, 159, 155) 10px)
}
.color-4 {
  background-image: repeating-linear-gradient(45deg, rgb(208, 187, 255) 0px, rgb(208, 187, 255) 10px)
}
.color-0-1 {
  background-image: repeating-linear-gradient(45deg, rgb(161, 201, 244) 0px, rgb(161, 201, 244) 10px, rgb(255, 180, 130) 10px, rgb(255, 180, 130) 20px)
}
.color-0-2 {
  background-image: repeating-linear-gradient(45deg, rgb(161, 201, 244) 0px, rgb(161, 201, 244) 10px, rgb(141, 229, 161) 10px, rgb(141, 229, 161) 20px)
}
.color-0-3 {
  background-image: repeating-linear-gradient(45deg, rgb(161, 201, 244) 0px, r

In [2]:
# CHANGE ME!
TASK_ID = 'youngest_over_35'
AUTHOR = 'will'

In [20]:
task = Task(
    id=TASK_ID,
    category="Basic",
    name="Youngest person over 35",
    description="Find the name of the youngest person older than 35, returning null if none exists.",
    plan=[
        Plan(id="filter", description="older than 35"),
        Plan(id="min", description="youngest person"),
        Plan(id="name", description="the name"),
        Plan(id="null", description="returning null")
    ],
    sample_input={
        "people": [
            {"age": 35, "name": "John"},
            {"age": 36, "name": "Mary"},
            {"age": 37, "name": "Jane"},
        ]
    },
    sample_output="Mary",
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [4]:
python_imperative_fused = replace(prototype,
    language='python-imperative',
    implementation='fused',
    source='''def youngest_over_35(people):
  youngest = None
  for person in people:
    if (person['age'] > 35 and 
       (youngest is None or person['age'] < youngest['age'])):
      youngest = person
  return youngest['name'] if youngest is not None else None''').load_plan()
python_imperative_fused.execute(task)
python_imperative_fused.save()

In [9]:
python_imperative_fused.widget(task)

Output()

CodeWidget(program='{"task": "youngest_over_35", "language": "python-imperative", "plan": {"min": [{"line": 1,…

In [10]:
python_imperative_separate = replace(prototype,
    language='python-imperative',
    implementation='separate',
    source='''def youngest_over_35(people):
  over_35 = []
  for person in people:
    if person['age'] > 35:
      over_35.append(person)
    
  youngest = None
  for person in over_35:
    if youngest is None or person['age'] < youngest['age']:
      youngest = person
  return youngest['name'] if youngest is not None else None''').load_plan()
python_imperative_separate.execute(task)
python_imperative_separate.save()

In [11]:
python_imperative_separate.widget(task)

Output()

CodeWidget(program='{"task": "youngest_over_35", "language": "python-imperative", "plan": {"filter": [{"line":…

In [12]:
python_functional = replace(prototype,
    language='python-functional',
    source='''def youngest_over_35(people):
  over_35 = [p for p in people if p['age'] > 35]
  youngest = min(over_35, default=None, key=lambda p: p['age'])
  return youngest['name'] if youngest is not None else None''').load_plan()
python_functional.execute(task)
python_functional.save()

In [13]:
python_functional.widget(task)

Output()

CodeWidget(program='{"task": "youngest_over_35", "language": "python-functional", "plan": {"filter": [{"line":…

In [14]:
python_pandas = replace(prototype,
    language='python-pandas',
    source='''def youngest_over_35(people):
  over_35 = people[people.age > 35]
  if len(over_35) == 0: 
    return None
  else:
    youngest = over_35.loc[over_35.age.idxmin()]
    return youngest['name']''').load_plan()
python_pandas.execute(task)
python_pandas.save()

In [15]:
python_pandas.widget(task)

Output()

CodeWidget(program='{"task": "youngest_over_35", "language": "python-pandas", "plan": {}, "source": "def young…

In [16]:
sql = replace(prototype,
    language='sql',
    source='''SELECT name
FROM people
WHERE age = (
  SELECT MIN(age)
  FROM people
  WHERE age > 35)
LIMIT 1''').load_plan()
sql.execute(task)
sql.save()

In [17]:
sql.widget(task)

Output()

CodeWidget(program='{"task": "youngest_over_35", "language": "sql", "plan": {}, "source": "SELECT name\\nFROM …

In [18]:
datalog = replace(prototype,
    language='datalog',
    source='''youngest_over_35(Name) :- 
  people(Age, Name), 
  Age = min Age : { people(Age, _), Age > 35 }.''').load_plan()
datalog.execute(task)
datalog.save()

In [19]:
datalog.widget(task)

Output()

CodeWidget(program='{"task": "youngest_over_35", "language": "datalog", "plan": {}, "source": "youngest_over_3…